In [25]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.express as px


In [2]:
path_data = "2024-barometre-consommation.xlsx"
df = pd.read_excel(path_data)

In [3]:
src_path = "2024-datamap.xlsx"
df_texts = pd.read_excel(src_path, sheet_name="TEXTS")

In [4]:
df_texts.columns = [c.strip().upper() for c in df_texts.columns]

if "TYPE" in df_texts.columns:
    type_norm = df_texts["TYPE"].astype(str).str.strip().str.upper()
    mask_title = type_norm.eq("TITLE")
    # ensure CODE column exists
    if "CODE" not in df_texts.columns:
        df_texts["CODE"] = pd.NA
    df_texts.loc[mask_title, "CODE"] = 0

if "CODE" in df_texts.columns:
    df_texts["CODE"] = pd.to_numeric(df_texts["CODE"], errors="coerce").astype("Int64")

df_texts = df_texts.drop(columns=["VALUE", "FORMULA", "TYPE"], errors="ignore")
out_path = Path("datamap_texts_clean.csv")
df_texts.to_csv(out_path, index=False, encoding="utf-8")

print(f"✅ Cleaned TEXTS saved: {out_path.resolve()}")
print(f"Shape: {df_texts.shape}")
print(df_texts.head(8))


✅ Cleaned TEXTS saved: C:\Users\emman\Documents\Cours\Data Analysis & Visualisation\Data Visualisation\Projet\datamap_texts_clean.csv
Shape: (2046, 4)
  HEADER TEXT                               NAME  CODE  \
0        TEXT  Baromètre de la consommation 2024     0   
1        TEXT                                 ID     0   
2        TEXT                    SYS_ELAPSEDTIME     0   
3        TEXT                               SEXE     0   
4        TEXT                               SEXE     1   
5        TEXT                               SEXE     2   
6        TEXT                                AGE     0   
7        TEXT                              QSEXE     0   

                                                FR:L  
0              Baromètre de la consommation 2024.sav  
1                                             Login:  
2                                       Elapsed Time  
3                              SEXE - Vous êtes... ?  
4                                           Un homm

In [5]:
empty_cols = df.columns[df.isna().all()].tolist()

print(f"🔍 Found {len(empty_cols)} completely empty columns:\n")
for col in empty_cols:
    print(" -", col)

df = df.drop(columns=empty_cols)
print(f"✅ Cleaned dataset: {df.shape[1]} columns remaining")

out_path = Path("dataset_clean.csv")
df.to_csv(out_path, index=False, encoding="utf-8")

🔍 Found 427 completely empty columns:

 - PI11
 - Q2_r10
 - Q2_r11
 - Q2BISB_r1_c1
 - Q2BISB_r1_c2
 - Q2BISB_r1_c3
 - Q2BISB_r2_c1
 - Q2BISB_r2_c2
 - Q2BISB_r2_c3
 - Q2BISB_r3_c1
 - Q2BISB_r3_c2
 - Q2BISB_r3_c3
 - Q2BISB_r4_c1
 - Q2BISB_r4_c2
 - Q2BISB_r4_c3
 - Q2BISB_r5_c1
 - Q2BISB_r5_c2
 - Q2BISB_r5_c3
 - Q2BISB_r6_c1
 - Q2BISB_r6_c2
 - Q2BISB_r6_c3
 - Q2BIS
 - Q2TER_1
 - Q2TER_2
 - Q2TER_3
 - Q2TER_4
 - Q2TER_5
 - Q2TER_6
 - Q2TER_7
 - Q2QUATR_r1
 - Q2QUATR_r2
 - AFQ8i1
 - AFQ8i2
 - Q8_r1
 - Q8_r2
 - Q3
 - Q4
 - Q5_r10
 - Q5_r11
 - Q6_r10
 - Q6_r11
 - QBOL1
 - QBOL2_1
 - QBOL2_2
 - QBOL2_3
 - QBOL2_4
 - QBOL2_5
 - QBOL2_6
 - QBOL2_7
 - QBOL2_8
 - QBOL2_9
 - QBOL2_10
 - QBOL2_11
 - QBOL2_12
 - QBOL3_1
 - QBOL3_2
 - QBOL3_3
 - QBOL3_4
 - QBOL3_5
 - QBOL3_6
 - QBOL3_7
 - QBOL3_8
 - QBOL3_9
 - QBOL3_10
 - QBOL3_11
 - QBOL3_12
 - QBOL3_13
 - QBOL5
 - QBOL6
 - QBOL7
 - QBOL8
 - QBOL9
 - QBOL10
 - QBOL11
 - QBOL12_1
 - QBOL12_2
 - QBOL12_3
 - QBOL13_1
 - QBOL13_2
 - QBOL13_3
 - QBOL13_4
 

In [6]:
# 1️⃣ Charger les fichiers
df_dict = pd.read_csv("datamap_texts_clean_filtered.csv")  # ton dictionnaire nettoyé

# 2️⃣ Normaliser les noms
df_dict.columns = [c.strip().upper() for c in df_dict.columns]
df_dict["NAME"] = df_dict["NAME"].astype(str).str.strip().str.upper()
df.columns = [c.strip().upper() for c in df.columns]

# 3️⃣ Nettoyer le dictionnaire
# enlever les doublons et lignes vides
df_dict = df_dict.dropna(subset=["NAME", "FR:L"]).drop_duplicates(subset=["NAME"], keep="first")

# 4️⃣ Créer le mapping complet NAME → FR:L
mapping = dict(zip(df_dict["NAME"], df_dict["FR:L"]))

# 5️⃣ Appliquer le renommage sur TOUTES les colonnes
def rename_with_fallback(col):
    col_upper = col.strip().upper()
    # correspondance exacte (pas de prefix matching ici)
    return mapping.get(col_upper, col)

df.columns = [rename_with_fallback(c) for c in df.columns]

print(f"✅ All {len(df.columns)} columns renamed using FR:L labels.")
print("🪶 Sample of renamed columns:")
for c in df.columns[:15]:
    print(" -", c)

# 6️⃣ Sauvegarde du dataset final renommé
df.to_csv("barometre_full_FR.csv", index=False, encoding="utf-8")
print("\n💾 Saved renamed dataset: barometre_full_FR.csv")


✅ All 1091 columns renamed using FR:L labels.
🪶 Sample of renamed columns:
 - Login:
 - Elapsed Time
 - SEXE - Vous êtes... ?
 - AGE - Quel âge avez-vous ?  Merci de noter votre âge dans le cadre ci-dessous :
 - QSEXE
 - RAGE1
 - RAGE2
 - RAGE3
 - AGGLOIFOP0
 - AGGLO
 - AGGLOIFOP2
 - TYPCOM
 - TAILCOM
 - DPT
 - REG

💾 Saved renamed dataset: barometre_full_FR.csv


In [7]:
df_dict_clean = df_texts[~df_texts["NAME"].isin(empty_cols)]
print(f"✅ Dictionary cleaned: {df_dict_clean.shape[0]} rows remaining")

# Sauvegarde du dictionnaire nettoyé
df_dict_clean.to_csv("datamap_texts_clean_filtered.csv", index=False, encoding="utf-8")
print("💾 Saved filtered dictionary: datamap_texts_clean_filtered.csv")

✅ Dictionary cleaned: 1964 rows remaining
💾 Saved filtered dictionary: datamap_texts_clean_filtered.csv


In [25]:
df = pd.read_csv("barometre_renamed.csv")

print(f"Dataset loaded: {df.shape}")
df.head(3)

Dataset loaded: (5041, 1091)


,Login:,Elapsed Time,SEXE - Vous êtes... ?,AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :,QSEXE,RAGE1,RAGE2,RAGE3,AGGLOIFOP0,AGGLO,...,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r1_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r2_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r3_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r4_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r5_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r6_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r7_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r8_c2,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r9_c2,Randomized Pages
0,IqW8n46nMmHQLInkVt-FLw**,267,1,52,1,5,4,3,UU02,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"108,111,110,109,112"
1,IqW8n46nMmGzI7dS4awivw**,227,1,56,1,5,4,3,UU08,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IqW8n46nMmHW6zBgU5OD6A**,579,2,40,2,4,3,2,UU08,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_dict = pd.read_csv("datamap_texts_clean_filtered.csv")

# === 2. Normaliser ===
df_dict.columns = [c.strip().upper() for c in df_dict.columns]
df_dict["NAME"] = df_dict["NAME"].astype(str).str.strip().str.upper()
df_dict["FR:L"] = df_dict["FR:L"].astype(str).str.strip()

# === 3. Construire un dictionnaire des correspondances NAME -> FR:L (pour trouver les colonnes FR correspondantes) ===
name_to_fr = df_dict.drop_duplicates(subset=["NAME"], keep="first").set_index("NAME")["FR:L"].to_dict()

# === 4. Créer les mappings de valeurs (code → label) ===
label_mappings = {}
for name in df_dict["NAME"].unique():
    sub = df_dict[df_dict["NAME"] == name].dropna(subset=["CODE", "FR:L"])
    if not sub.empty:
        mapping = dict(zip(sub["CODE"].astype(str), sub["FR:L"]))
        label_mappings[name] = mapping

print(f"✅ {len(label_mappings)} variable-level mappings built.")

# === 5. Appliquer les mappings aux colonnes correspondantes dans le df ===
applied = 0
for name, mapping in label_mappings.items():
    fr_label = name_to_fr.get(name)  # nom français de la colonne dans df
    if fr_label and fr_label in df.columns:
        # convertir toutes les valeurs en string pour le remplacement (plus robuste)
        df[fr_label] = df[fr_label].astype(str).replace(mapping)
        applied += 1

print(f"🔠 Label mappings applied to {applied} columns successfully.")

# === 6. Sauvegarde du dataset final ===
df.to_csv("barometre_full_FR_labeled.csv", index=False, encoding="utf-8")
print("💾 Saved labeled dataset: barometre_full_FR_labeled.csv")

# === 7. Vérification rapide ===
print("\n🪶 Example of labeled values:")
print(df.head(5))

✅ 282 variable-level mappings built.
🔠 Label mappings applied to 121 columns successfully.
💾 Saved labeled dataset: barometre_full_FR_labeled.csv

🪶 Example of labeled values:
                     Login: Elapsed Time SEXE - Vous êtes... ?  \
0  IqW8n46nMmHQLInkVt-FLw**          267              Un homme   
1  IqW8n46nMmGzI7dS4awivw**          227              Un homme   
2  IqW8n46nMmHW6zBgU5OD6A**          579             Une femme   
3  IqW8n46nMmGG3ojBPpTdVg**          390              Un homme   
4  IqW8n46nMmF6Y65ClZzfMA**          315             Une femme   

   AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :  \
0                                                 52                                
1                                                 56                                
2                                                 40                                
3                                                 49                                
4 

In [29]:
print(f"Initial shape: {df.shape}")

# --- 1.1 Colonnes dupliquées (même contenu)
duplicated_cols = df.T.duplicated(keep="first")
duplicate_names = df.columns[duplicated_cols].tolist()
print(f"🪞 Duplicate columns detected: {len(duplicate_names)}")
if duplicate_names:
    print(duplicate_names[:15])

# --- 1.2 Colonnes constantes (une seule valeur unique)
constant_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
print(f"⚙️ Constant columns detected: {len(constant_cols)}")
if constant_cols:
    print(constant_cols[:15])

# --- 1.3 Colonnes quasi vides (plus de 95% de NaN)
mostly_empty = [c for c in df.columns if df[c].isna().mean() > 0.95]
print(f"💨 Mostly empty columns (>95% NaN): {len(mostly_empty)}")
if mostly_empty:
    print(mostly_empty[:15])

# --- 1.4 Colonnes très longues (textuelles inutiles ou commentaires)
long_text_cols = [c for c in df.columns if df[c].astype(str).map(len).mean() > 100]
print(f"📜 Possibly verbose text columns (>100 char avg): {len(long_text_cols)}")
if long_text_cols:
    print(long_text_cols[:10])

Initial shape: (5041, 1091)
🪞 Duplicate columns detected: 0
⚙️ Constant columns detected: 0
💨 Mostly empty columns (>95% NaN): 480
['Q6 - Pour chacun des produits ou services culturels suivants, à quelle fréquence les consommez-vous de façon illégale… Une réponse par ligne. Vous consommez sur Internet de façon illégale…_r4', 'Q6 - Pour chacun des produits ou services culturels suivants, à quelle fréquence les consommez-vous de façon illégale… Une réponse par ligne. Vous consommez sur Internet de façon illégale…_r5', 'Q6 - Pour chacun des produits ou services culturels suivants, à quelle fréquence les consommez-vous de façon illégale… Une réponse par ligne. Vous consommez sur Internet de façon illégale…_r6', 'Q6 - Pour chacun des produits ou services culturels suivants, à quelle fréquence les consommez-vous de façon illégale… Une réponse par ligne. Vous consommez sur Internet de façon illégale…_r7', 'Q6 - Pour chacun des produits ou services culturels suivants, à quelle fréquence les co

In [30]:
# 🔹 Supprimer les colonnes quasi vides
df_cleaned = df.drop(columns=mostly_empty)
print(f"✅ After dropping empty columns: {df_cleaned.shape}")

# 🔹 Sauvegarder le dataset nettoyé
df_cleaned.to_csv("barometre_clean_selected.csv", index=False, encoding="utf-8")
print("💾 Saved: barometre_clean_selected.csv")


✅ After dropping empty columns: (5041, 611)
💾 Saved: barometre_clean_selected.csv


In [33]:
(df_cleaned.isna().mean().sort_values().head(20) * 100).round(1)


Login:                                                                                                                                                        0.0
QBU9 - A quelle fréquence consommez-vous des contenus culturels et sportifs sur votre smartphone ou votre tablette de manière illégale ?                      0.0
QBU100B - Vous avez indiqué recourir à l’IPTV pour consommer illégalement certains produits culturels dématérialisés. Avez-vous accès à l’IPTV de façon… ?    0.0
QBU100c2 -                                                                                                                                                    0.0
QBU100D - Avez-vous accès à l’IPTV… ?                                                                                                                         0.0
AFQBU11i1                                                                                                                                                     0.0
AFQBU11i2                   

In [35]:
df_cleaned.columns

Index(['Login:', 'Elapsed Time', 'SEXE - Vous êtes... ?',
       'AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :',
       'QSEXE', 'RAGE1', 'RAGE2', 'RAGE3', 'AGGLOIFOP0', 'AGGLO',
       ...
       'RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r2_c1',
       'RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r3_c1',
       'RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r4_c1',

In [37]:
# ============================================================
# CLEAN SEMANTIC DUPLICATES (AGE, SEXE, etc.)
# ============================================================

cols_to_drop = []

# 🔹 1. Remove redundant derived variables for age and gender
age_related = [c for c in df_cleaned.columns if re.search(r"\bRAGE|QSEXE|AGE\d|AGE_|R_AGE\b", c, flags=re.IGNORECASE)]
sex_related = [c for c in df_cleaned.columns if re.search(r"\bQSEXE|Q_SEXE|SEXE\d|SEXE_|RSEXE|R_SEXE\b", c, flags=re.IGNORECASE)]
cols_to_drop += age_related + sex_related

# 🔹 2. Remove control / technical columns
technical_cols = [c for c in df_cleaned.columns if re.match(r"(AFRS|AFQ|SYS_|CELLULE|CEL)", c)]
cols_to_drop += technical_cols

# 🔹 3. Remove repetitive label fragments ("_R1", "_R2", etc.) if they belong to multi-line questions
repeated_cols = [c for c in df_cleaned.columns if re.search(r"_R\d+$", c)]
cols_to_drop += repeated_cols

# 🔹 4. Drop duplicates safely
cols_to_drop = list(set(cols_to_drop))
print(f"🧹 Dropping {len(cols_to_drop)} redundant / routing columns...")

df_semantic = df_cleaned.drop(columns=[c for c in cols_to_drop if c in df_cleaned.columns])
print(f"✅ Dataset after semantic cleanup: {df_semantic.shape}")

# Save cleaned dataset
df_semantic.to_csv("barometre_clean_semantic.csv", index=False, encoding="utf-8")
print("💾 Saved: barometre_clean_semantic.csv")


🧹 Dropping 51 redundant / routing columns...
✅ Dataset after semantic cleanup: (5041, 560)
💾 Saved: barometre_clean_semantic.csv


In [39]:
df_semantic.head()

,Login:,Elapsed Time,SEXE - Vous êtes... ?,AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :,AGGLOIFOP0,AGGLO,AGGLOIFOP2,TYPCOM,TAILCOM,DPT,...,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r2_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r3_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r4_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r5_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r6_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r7_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r8_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r9_c1,RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_r2_c2,Randomized Pages
0,IqW8n46nMmHQLInkVt-FLw**,267,Un homme,52,UU02,CC2,MoinsDe100000Habitants,Rural,De3500a4999hab,OISE,...,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"108,111,110,109,112"
1,IqW8n46nMmGzI7dS4awivw**,227,Un homme,56,UU08,CC5,PlusDe100000Habitants,BanlieueNVSup,De100000habEtPlus,HAUTSDESEINE,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,nan
2,IqW8n46nMmHW6zBgU5OD6A**,579,Une femme,40,UU08,CC5,PlusDe100000Habitants,BanlieueNVModeste,De50000a99999hab,SEINESAINTDENIS,...,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,nan
3,IqW8n46nMmGG3ojBPpTdVg**,390,Un homme,49,UU02,CC2,MoinsDe100000Habitants,VilleIsolee,De5000a8999hab,YONNE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
4,IqW8n46nMmF6Y65ClZzfMA**,315,Une femme,50,UU07,CC4,PlusDe100000Habitants,VilleCentre,De100000habEtPlus,MAINEETLOIRE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan


In [41]:


# Suppose ton DataFrame s'appelle df (dataset original ordonné)
df_clean = df_semantic.copy()

# Fonction pour extraire la racine d’une variable (ex: AGGLOIFOP0 → AGGLO)
def base_name(col):
    # On simplifie : on retire les suffixes typiques
    col = re.sub(r"(IFOP\d*|BIS\d*|TER\d*|[0-9]+$|_[A-Z0-9]+$)", "", col)
    return col.rstrip("_")

# Dictionnaire pour suivre la première occurrence
seen = {}
cols_to_keep = []
cols_to_drop = []

for col in df_clean.columns:
    root = base_name(col)
    if root not in seen:
        # Première fois qu'on voit cette famille → on garde
        seen[root] = col
        cols_to_keep.append(col)
    else:
        # Famille déjà vue → redondance
        cols_to_drop.append(col)

print(f"🔁 Redundant variable families detected: {len(seen)}")
print(f"✅ Keeping {len(cols_to_keep)} columns, dropping {len(cols_to_drop)} redundant variants.")

# Crée un DataFrame nettoyé
df_primary = df_clean[cols_to_keep].copy()
print(f"📏 Shape after cleanup: {df_primary.shape}")

# (Optionnel) Sauvegarde
df_primary.to_csv("barometre_clean_primary.csv", index=False, encoding="utf-8")
print("💾 Saved: barometre_clean_primary.csv")


🔁 Redundant variable families detected: 151
✅ Keeping 151 columns, dropping 409 redundant variants.
📏 Shape after cleanup: (5041, 151)
💾 Saved: barometre_clean_primary.csv


In [ ]:
df_primary.head()

In [43]:
pd.DataFrame(df_primary.columns, columns=["ColumnName"]).to_csv("columns_remaining.csv", index=False)


In [45]:
n_samples = 5

# Sélectionne 5 lignes aléatoires pour l’aperçu
df_sample = df_primary.sample(n=n_samples, random_state=42).reset_index(drop=True)

# Crée un export Excel clair avec les 5 lignes sous chaque colonne
output_file = "barometre_preview.xlsx"
df_sample.to_excel(output_file, index=False, engine='openpyxl')

print(f"✅ Excel file created: {output_file}")
print(f"Contains {len(df_sample.columns)} columns and {len(df_sample)} sample rows.")

✅ Excel file created: barometre_preview.xlsx
Contains 151 columns and 5 sample rows.


In [49]:
df_primary.to_excel("baro.xlsx", index=False, engine='openpyxl')

In [51]:
missing_ratio = df_primary.isna().mean().sort_values(ascending=False) * 100

# Transforme en DataFrame pour une meilleure lisibilité
missing_summary = (
    pd.DataFrame({
        "Column": missing_ratio.index,
        "Missing (%)": missing_ratio.values.round(2)
    })
)

print(f"✅ Computed missing value percentages for {len(missing_summary)} columns.")

# Affiche les 10 premières colonnes les plus vides
display(missing_summary.head(10))

# Sauvegarde complète (les 140 colonnes)
missing_summary.to_excel("barometre_missing_summary.xlsx", index=False, engine="openpyxl")
print("💾 Excel file saved: barometre_missing_summary.xlsx")

✅ Computed missing value percentages for 151 columns.


,Column,Missing (%)
0,RS16BIS - À qui appartiennent ces codes d’accè...,94.56
1,Q6 - Pour chacun des produits ou services cult...,94.39
2,RS16BIS - À qui appartiennent ces codes d’accè...,94.23
3,"- Pour consommer illégalement des séries TV , ...",93.99
4,RS16BIS - À qui appartiennent ces codes d’accè...,93.73
5,"QBU5a - Et au cours des 12 derniers mois, sur ...",93.71
6,"- Lorsque vous écoutez des podcasts en ligne ,...",93.67
7,QBOL15TER - Quels services ou plateformes préf...,93.59
8,QBU11 - À quelle fréquence utilisez-vous les m...,93.16
9,RS16BIS - À qui appartiennent ces codes d’accè...,93.10


💾 Excel file saved: barometre_missing_summary.xlsx


In [53]:
# 🔹 On part de ton DataFrame nettoyé, par exemple df_primary
df_clean = df_primary.copy()

# ======================================================
# 1️⃣ Supprimer les colonnes non pertinentes
# ======================================================
cols_to_remove = [
    "RECPPIA", "STCB", "STCC", "STATU1",
    "SITC - ACTUELLEMENT, QUELLE EST LA SITUATION DU CHEF DE FAMILLE DE VOTRE FOYER ?",
    "PCFA - PLUS PRÉCISÉMENT, QUELLE EST SA PROFESSION PRINCIPALE OU, SI ELLE EST RETRAITÉE OU AU CHÔMAGE, LA DERNIÈRE PROFESSION PRINCIPALE QU'ELLE A EXERCÉE ?",
    "PCF1",
    "- QUEL EST L'ÂGE DE VOTRE / VOS ENFANT(S) VIVANT DANS VOTRE FOYER ? VOUS POUVEZ SÉLECTIONNER PLUSIEURS RÉPONSES._1",
    "Q1_1",
    "- VOICI PLUSIEURS RAISONS QUI PEUVENT INCITER À CONSOMMER DE FAÇON LEGALE DES PRODUITS CULTURELS SUR INTERNET. INDIQUEZ CELLES QUE VOUS TROUVEZ PERSONNELLEMENT LES PLUS MOTIVANTES POUR CONSOMMER DE FAÇON LÉGALE . VOUS POUVEZ SÉLECTIONNER_1",
    "- LORSQUE VOUS ÉCOUTEZ DE LA MUSIQUE OU REGARDEZ DES VIDÉOS CLIPS EN LIGNE , QUELS SERVICES OU PLATEFORMES CONSULTEZ-VOUS ? VOUS POUVEZ SÉLECTIONNER PLUSIEURS RÉPONSES._1",
    "- LORSQUE VOUS REGARDEZ DES FILMS EN LIGNE , QUELS SERVICES OU PLATEFORMES CONSULTEZ-VOUS ? VOUS POUVEZ SÉLECTIONNER PLUSIEURS RÉPONSES._1",
    "QBOL14TER - QUELS SERVICES OU PLATEFORMES PRÉFÉREZ-VOUS CONSULTER POUR REGARDER DES FILMS EN LIGNE ? MERCI DE LES CLASSER PAR ORDRE DE PRÉFÉRENCE._1",
    "- LORSQUE VOUS REGARDEZ DES SÉRIES EN LIGNE , QUELS SERVICES OU PLATEFORMES CONSULTEZ-VOUS ? VOUS POUVEZ SÉLECTIONNER PLUSIEURS RÉPONSES._1",
    "QBOL15TER - QUELS SERVICES OU PLATEFORMES PRÉFÉREZ-VOUS CONSULTER POUR REGARDER DES SÉRIES EN LIGNE ? MERCI DE LES CLASSER PAR ORDRE DE PRÉFÉRENCE._1"
]

df_clean = df_clean.drop(columns=[c for c in cols_to_remove if c in df_clean.columns])
print(f"🧹 Removed {len(cols_to_remove)} non-relevant columns.")

# ======================================================
# 2️⃣ Fusionner les questions à choix multiples (ex: QBOL21)
# ======================================================

# Détection automatique des familles de colonnes multiples
multi_prefixes = {}
for col in df_clean.columns:
    match = re.match(r"([A-Z]+[A-Z0-9]+)_\d+$", col)
    if match:
        root = match.group(1)
        multi_prefixes.setdefault(root, []).append(col)

print(f"🔍 Detected {len(multi_prefixes)} multi-choice questions.")

# Fusion : on combine toutes les réponses cochées en une seule colonne texte
for root, subcols in multi_prefixes.items():
    df_clean[root + "_COMBINED"] = df_clean[subcols] \
        .astype(str) \
        .replace({"nan": None, "0": None, "": None}) \
        .apply(lambda row: ", ".join([x for x in row if x not in [None, "0"]]), axis=1)
    
    # Supprime les colonnes éclatées
    df_clean = df_clean.drop(columns=subcols)

print(f"✅ Merged {len(multi_prefixes)} multi-choice question groups.")
print(f"📊 New shape: {df_clean.shape}")

# ======================================================
# 3️⃣ Sauvegarde du dataset nettoyé
# ======================================================
df_clean.to_csv("barometre_clean_final.csv", index=False, encoding="utf-8")
print("💾 Saved: barometre_clean_final.csv")


🧹 Removed 15 non-relevant columns.
🔍 Detected 0 multi-choice questions.
✅ Merged 0 multi-choice question groups.
📊 New shape: (5041, 145)
💾 Saved: barometre_clean_final.csv


In [55]:

df_clean = df_primary.copy()

# ============================================================
# 1️⃣ Détecter les familles de colonnes multi-réponses (rX_cY)
# ============================================================
pattern = r"(.+?)_r\d+_c\d+$"
multi_groups = {}

for col in df_clean.columns:
    match = re.match(pattern, col)
    if match:
        root = match.group(1).strip()
        multi_groups.setdefault(root, []).append(col)

print(f"🔍 Detected {len(multi_groups)} multi-response question groups.")

# ============================================================
# 2️⃣ Fusionner chaque groupe
# ============================================================
for root, subcols in multi_groups.items():
    df_clean[root + "_COMBINED"] = (
        df_clean[subcols]
        .astype(str)
        .replace({"nan": None, "0": None, "": None})
        .apply(lambda row: ", ".join([x for x in row if x not in [None, "0"]]), axis=1)
    )
    df_clean = df_clean.drop(columns=subcols)

print(f"✅ Merged {len(multi_groups)} groups successfully.")
print(f"📊 New shape: {df_clean.shape}")

# ============================================================
# 3️⃣ Sauvegarde du dataset nettoyé
# ============================================================
df_clean.to_csv("barometre_clean_final.csv", index=False, encoding="utf-8")
print("💾 Saved: barometre_clean_final.csv")

# (Optionnel) affichage d’un aperçu
if multi_groups:
    print("\n✨ Example of merged question:")
    sample_key = list(multi_groups.keys())[0]
    display(df_clean[[sample_key + "_COMBINED"]].head())


🔍 Detected 9 multi-response question groups.
✅ Merged 9 groups successfully.
📊 New shape: (5041, 80)
💾 Saved: barometre_clean_final.csv

✨ Example of merged question:


,"QBOL21 - Lorsque vous consommez les produits culturels suivants sur les réseaux sociaux , quelles plateformes consultez-vous ? Vous pouvez sélectionner plusieurs réponses par colonne. Musique / vidéos clips_COMBINED"
0,
1,
2,
3,
4,


In [57]:


# ======================================================
# 1️⃣ Charger les données
# ======================================================
df = df_primary.copy()
df_dict = pd.read_excel("2024-datamap.xlsx", sheet_name="TEXTS")

# Nettoyage des noms de colonnes du dictionnaire
df_dict.columns = [c.strip().upper() for c in df_dict.columns]
df_dict = df_dict[["NAME", "CODE", "FR:L"]].dropna(subset=["NAME", "FR:L"])

# Assure-toi que CODE est bien texte pour les correspondances
df_dict["CODE"] = df_dict["CODE"].astype(str).str.replace(".0", "", regex=False)

print(f"📘 Dictionary loaded: {len(df_dict)} entries.")

# ======================================================
# 2️⃣ Identifier les groupes multi-réponses (_rX_cY)
# ======================================================
pattern = r"(.+?)_r(\d+)_c\d+$"
multi_groups = {}

for col in df.columns:
    match = re.match(pattern, col)
    if match:
        root = match.group(1).strip()
        code = match.group(2).strip()
        multi_groups.setdefault(root, []).append((col, code))

print(f"🔍 Detected {len(multi_groups)} multi-response question groups.")

# ======================================================
# 3️⃣ Fusionner avec labels à partir du dictionnaire
# ======================================================
for root, items in multi_groups.items():
    subcols = [c for c, _ in items]
    
    # Récupère les correspondances code → label depuis df_dict
    label_map = (
        df_dict[df_dict["NAME"].str.contains(root, case=False, na=False)]
        .set_index("CODE")["FR:L"]
        .to_dict()
    )
    
    # Construit les combinaisons lisibles
    def combine_labels(row):
        selected = []
        for col, code in items:
            val = row[col]
            if pd.notna(val) and str(val).strip() not in ["0", "nan", ""]:
                label = label_map.get(str(code))
                if label:
                    selected.append(label)
                else:
                    selected.append(f"Code {code}")
        return ", ".join(selected)
    
    # Crée la nouvelle colonne combinée
    df[root + "_COMBINED"] = df.apply(combine_labels, axis=1)
    
    # Supprime les sous-colonnes
    df = df.drop(columns=subcols)

print(f"✅ Merged {len(multi_groups)} groups with label translation.")
print(f"📊 New shape: {df.shape}")

# ======================================================
# 4️⃣ Sauvegarde du dataset propre
# ======================================================
df.to_csv("barometre_clean_with_labels.csv", index=False, encoding="utf-8")
print("💾 Saved: barometre_clean_with_labels.csv")

# (Optionnel) aperçu d’un exemple
if multi_groups:
    example_key = list(multi_groups.keys())[0]
    print(f"\n✨ Example merged question: {example_key}")
    display(df[[example_key + "_COMBINED"]].head())


📘 Dictionary loaded: 2046 entries.
🔍 Detected 9 multi-response question groups.


C:\Users\emman\AppData\Local\Temp\ipykernel_51020\1337539473.py:42: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_dict[df_dict["NAME"].str.contains(root, case=False, na=False)]


✅ Merged 9 groups with label translation.
📊 New shape: (5041, 80)
💾 Saved: barometre_clean_with_labels.csv

✨ Example merged question: QBOL21 - Lorsque vous consommez les produits culturels suivants sur les réseaux sociaux , quelles plateformes consultez-vous ? Vous pouvez sélectionner plusieurs réponses par colonne. Musique / vidéos clips


,"QBOL21 - Lorsque vous consommez les produits culturels suivants sur les réseaux sociaux , quelles plateformes consultez-vous ? Vous pouvez sélectionner plusieurs réponses par colonne. Musique / vidéos clips_COMBINED"
0,
1,
2,
3,
4,


In [65]:
df.to_excel("new.xlsx", index=False, engine='openpyxl')

In [67]:


print(f"📊 Initial shape: {df.shape}")

# =====================================================
# 1️⃣ Retirer les colonnes explicitement mentionnées
# =====================================================
cols_to_drop = [
    "PI12",
    "STCB",
    "STCC",
    "STATU1",
    "SITC - Actuellement, quelle est la situation du chef de famille de votre foyer ?",
    "PCFA - Plus précisément, quelle est sa profession principale ou, si elle est retraitée ou au chômage, la dernière profession principale qu'elle a exercée ?",
    "QRS6"
]

cols_to_drop_existing = [c for c in cols_to_drop if c in df.columns]
df = df.drop(columns=cols_to_drop_existing)
print(f"🧹 Removed {len(cols_to_drop_existing)} manually listed columns.")

# =====================================================
# 2️⃣ Supprimer les colonnes avec trop de NaN
# =====================================================
# Seuil de tolérance : garde les colonnes avec au moins 30% de données valides
threshold = 0.3
valid_ratio = df.notna().mean()
low_info_cols = valid_ratio[valid_ratio < threshold].index.tolist()

df = df.drop(columns=low_info_cols)
print(f"🚮 Removed {len(low_info_cols)} columns with < {int(threshold*100)}% valid data.")

# =====================================================
# 3️⃣ Sauvegarde
# =====================================================
df.to_csv("barometre_clean_final_filtered.csv", index=False, encoding="utf-8")
print(f"💾 Saved: barometre_clean_final_filtered.csv")
print(f"✅ Final shape: {df.shape}")

# Aperçu du taux de complétude global
completeness = (df.notna().mean() * 100).round(1)
print("\n📈 Top 10 columns by completeness:")
print(completeness.sort_values(ascending=False).head(10))


📊 Initial shape: (5041, 80)
🧹 Removed 7 manually listed columns.
🚮 Removed 18 columns with < 30% valid data.
💾 Saved: barometre_clean_final_filtered.csv
✅ Final shape: (5041, 55)

📈 Top 10 columns by completeness:
Login:                                                                                                                                                        100.0
AFFi1                                                                                                                                                         100.0
QBU2BIS2 -                                                                                                                                                    100.0
QBU8 - Globalement, depuis combien de temps vous arrive-t-il de consommer des contenus culturels et sportifs dématérialisés de manière illégale ?             100.0
QBU9 - A quelle fréquence consommez-vous des contenus culturels et sportifs sur votre smartphone ou votre tablette de manière illé

In [71]:
df = pd.read_csv("barometre_clean_final_filtered.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5041 entries, 0 to 5040
Data columns (total 55 columns):
 #   Column                                                                                                                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                       --------------  -----  
 0   Login:                                                                                                                                                                                                                                                       5041 non-null   object 
 1   Elapsed Time                              

In [73]:
cols_tech = ["Login:", "Elapsed Time", "Randomized Questions", "Randomized Pages"]
df = df.drop(columns=[c for c in cols_tech if c in df.columns])
print(f"🧹 Dropped {len(cols_tech)} technical columns.")

# 2️⃣ Supprimer les colonnes trop vides
min_valid_ratio = 0.10  # au moins 10 % de réponses
ratio = df.notna().mean()
low_cols = ratio[ratio < min_valid_ratio].index.tolist()
df = df.drop(columns=low_cols)
print(f"🚮 Removed {len(low_cols)} low-information columns (<10% filled).")

# 3️⃣ Sauvegarder la version finale
df.to_csv("barometre_final_for_analysis.csv", index=False)
print(f"💾 Saved: barometre_final_for_analysis.csv")
print(f"✅ Final dataset shape: {df.shape}")

# 4️⃣ Aperçu du taux de remplissage
fill_ratio = (df.notna().mean() * 100).sort_values(ascending=False)
print("\n📊 Column completeness (%):")
print(fill_ratio)

🧹 Dropped 4 technical columns.
🚮 Removed 8 low-information columns (<10% filled).
💾 Saved: barometre_final_for_analysis.csv
✅ Final dataset shape: (5041, 43)

📊 Column completeness (%):
SEXE - Vous êtes... ?                                                                                                                                                                                                                                          100.000000
STATUT - Au sein de votre foyer, quelle est votre situation ?                                                                                                                                                                                                  100.000000
RS8 - Et vous arrive-t-il de faire des réglages de DNS ?                                                                                                                                                                                                       100.000000
RS15BIS - Certai

In [75]:
min_valid_ratio = 0.20
valid_ratio = df.notna().mean()
low_cols = valid_ratio[valid_ratio < min_valid_ratio].index.tolist()

df = df.drop(columns=low_cols)
print(f"🚮 Removed {len(low_cols)} low-filled columns (<20% data).")
print(f"✅ Final shape: {df.shape}")

df.to_csv("barometre_final_clean.csv", index=False, encoding="utf-8")
print("💾 Saved: barometre_final_clean.csv")

# Afficher les colonnes supprimées
print("\n❌ Removed columns:")
for c in low_cols:
    print(f" - {c}")

🚮 Removed 3 low-filled columns (<20% data).
✅ Final shape: (5041, 40)
💾 Saved: barometre_final_clean.csv

❌ Removed columns:
 - TITC - Dans la fonction publique, êtes-vous... ?
 - QBU8 - Globalement, depuis combien de temps vous arrive-t-il de consommer des contenus culturels et sportifs dématérialisés de manière illégale ?
 - QBU9 - A quelle fréquence consommez-vous des contenus culturels et sportifs sur votre smartphone ou votre tablette de manière illégale ?


In [77]:


# Display config
pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")

# Load the dataset
df = pd.read_csv("barometre_final_clean.csv")
print(f"✅ Dataset loaded: {df.shape[0]} rows × {df.shape[1]} columns")

# Quick preview
display(df.head(3))


✅ Dataset loaded: 5041 rows × 40 columns


,SEXE - Vous êtes... ?,AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :,AGGLOIFOP0,TYPCOM,TAILCOM,DPT,REG,"SITI - Actuellement, quelle est votre situation ?","PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d",RECPPIA,"STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal.",STCA,"STATUT - Au sein de votre foyer, quelle est votre situation ?",PCF1,FOYER - De combien de personnes se compose votre foyer y compris vous-même ?,"ENF - Au total, combien y a-t-il d’enfants de moins de 18 ans dans votre foyer ?",- Quel est l'âge de votre / vos enfant(s) vivant dans votre foyer ? Vous pouvez sélectionner plusieurs réponses._1,"RS6 - A quelle fréquence utilisez-vous Internet ou des applications, quels que soient le lieu d’utilisation et l’appareil de connexion ?",Q1_1,Q2 - À quelle fréquence consommez-vous sur Internet chacun des produits ou services culturels dématérialisés suivants ? Une réponse par ligne. Vous consommez sur Internet …_r1,"Q5 - Plus précisément, pour chacun des produits ou services culturels suivants, diriez-vous que vous les consommez… Une réponse par ligne. sur Internet …_r1","Q7 - Concernant votre consommation de biens culturels dématérialisés, diriez-vous qu’aujourd’hui :",QBU1 - Vous nous avez dit consommer de façon dématérialisée les contenus culturels et sportifs suivants. Veuillez indiquer pour chacun d’eux si vous les consommez gratuitement ou de façon payante. On parle toujours de contenus culturels et spor_r1,AFFi1,"QBU2 - De façon générale, quel montant dépensez-vous en moyenne chaque mois pour votre consommation de [% ListLabel(Q1List,AFFi1) %] [% ListLabel(Q1List,AFFi2) %] [% ListLabel(Q1List,AFFi3) %] [% ListLabel(Q1List,AFFi4) %] [% ListLabel(Q1List,AFFi5","- Utilisez-vous des applications « crackées » que vous avez téléchargées sur des stores d’applications alternatifs (comme AppValley ou Tutuapp par exemple) ou via des APKs, permettant l’accès à des offres payantes sans payer ? Vou_1","QBU12 - Utilisez-vous des logiciels, des applications ou des sites internet permettant de convertir des contenus consultés en streaming (films, séries, musique vus sur une plateforme) en un contenu à télécharger (qui permettent par exemple de conv_r1","- Parmi les équipements suivants, le(s)quel(s) possédez-vous à titre personnel ? Vous pouvez sélectionner plusieurs réponses._1",RS8 - Et vous arrive-t-il de faire des réglages de DNS ?,- A quelle(s) chaîne(s) payante(s) avez-vous accès ? Vous pouvez sélectionner plusieurs réponses._1,- A quels services ou applications de musique payants avez-vous accès (y compris de manière associée à un forfait téléphonique) ? Vous pouvez sélectionner plusieurs réponses._1,- A quel(s) service(s) de vidéo à la demande par abonnement payant avez-vous accès? Vous pouvez sélectionner plusieurs réponses._1,RS15BIS - Certains services de vidéo à la demande par abonnement proposent désormais des formules d’abonnement moins chères mais incluant de la publicité. Seriez-vous prêt à souscrire à une offre avec publicité pour réduire le coût de l’,DISPQUAL,"RS7BIS - Au cours des 12 derniers mois, avez-vous utilisé au moins un VPN à titre personnel ?","QBU5a - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","QBU5b - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","RS12BIS - Avez-vous accès aux fournisseurs de services payants suivants ? Attention, nous parlons ici des offres auxquelles vous

In [79]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5041 entries, 0 to 5040
Data columns (total 40 columns):
 #   Column                                                                                                                                                                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                                       --------------  -----  
 0   SEXE - Vous êtes... ?                                                                                                                                                                                                                                        5041 non-null   object 
 1   AGE - Quel âge avez-vous ? Merci de noter 

In [81]:
dupes = df.duplicated().sum()
print(f"🧩 Duplicate rows: {dupes}")


🧩 Duplicate rows: 0


In [83]:
missing = df.isna().mean().sort_values(ascending=False) * 100
print("📉 Missing values per column (%):")
display(missing)


📉 Missing values per column (%):


RS16BIS - À qui appartiennent ces codes d’accès extérieurs à votre foyer que vous utilisez ? Vous pouvez sélectionner plusieurs réponses par ligne. Pour_COMBINED                                                                                              73.279111
- A quelle(s) chaîne(s) payante(s) avez-vous accès ? Vous pouvez sélectionner plusieurs réponses._1                                                                                                                                                            66.117834
- Quel est l'âge de votre / vos enfant(s) vivant dans votre foyer ? Vous pouvez sélectionner plusieurs réponses._1                                                                                                                                             60.583218
- A quels services ou applications de musique payants avez-vous accès (y compris de manière associée à un forfait téléphonique) ? Vous pouvez sélectionner plusieurs réponses._1                             

In [85]:
const_cols = [c for c in df.columns if df[c].nunique(dropna=True) <= 1]
print(f"⚠️ Constant columns: {len(const_cols)}")
if const_cols:
    display(const_cols)


⚠️ Constant columns: 0


In [87]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"🔢 Numeric columns: {len(num_cols)} — {num_cols}")

df[num_cols].describe().T


🔢 Numeric columns: 18 — ['AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :', 'PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d', 'STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal.', "- Quel est l'âge de votre / vos enfant(s) vivant dans votre foyer ? Vous pouvez sélectionner plusieurs réponses._1", 'Q1_1', 'Q2 - À quelle fréquence consommez-vous sur Internet chacun des produits ou services culturels dématérialisés suivants ? Une réponse par ligne. Vous consommez sur Internet …_r1', 'Q5 - Plus précisément, pour chacun des produits ou services culturels suivants, diriez-vous que vous les consommez… Une réponse par ligne. sur Internet …_r1', 'Q7 - Concernant votre consommation de biens cultur

,count,mean,std,min,25%,50%,75%,max
AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :,5041.0,45.413013,16.972434,15.0,32.0,46.0,59.0,98.0
"PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d",4223.0,15.658773,7.170147,1.0,9.0,17.0,21.0,30.0
"STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal.",3179.0,3.881095,2.013567,1.0,3.0,3.0,5.0,10.0
- Quel est l'âge de votre / vos enfant(s) vivant dans votre foyer ? Vous pouvez sélectionner plusieurs réponses._1,1987.0,0.089079,0.284929,0.0,0.0,0.0,0.0,1.0
Q1_1,5041.0,0.575878,0.494258,0.0,0.0,1.0,1.0,1.0
Q2 - À quelle fréquence consommez-vous sur Internet chacun des produits ou services culturels dématérialisés suivants ? Une réponse par ligne. Vous consommez sur Internet …_r1,2903.0,1.588701,0.772750,1.0,1.0,1.0,2.0,4.0
"Q5 - Plus précisément, pour chacun des produits ou services culturels suivants, diriez-vous que vous les consommez… Une réponse par ligne. sur Internet …_r1",2903.0,1.146745,0.507520,1.0,1.0,1.0,1.0,5.0
"Q7 - Concernant votre consommation de biens culturels dématérialisés, diriez-vous qu’aujourd’hui :",4599.0,1.748424,0.604450,1.0,1.0,2.0,2.0,4.0
QBU1 - Vous nous avez dit consommer de façon dématérialisée les contenus culturels et sportifs suivants. Veuillez indiquer pour chacun d’eux si vous les consommez gratuitement ou de façon payante. On parle toujours de contenus culturels et spor_r1,2049.0,1.902879,1.388020,1.0,1.0,1.0,3.0,5.0
AFFi1,3541.0,42.292290,48.396825,1.0,1.0,1.0,99.0,99.0


In [89]:
df.to_excel("final.xlsx", index=False, engine='openpyxl')

In [91]:



print(f"Initial shape: {df.shape}")

# 1️⃣ Remove redundant columns
cols_to_drop = [
    "PCF1", "Q1_1",
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
print(f"🧹 Dropped {len(cols_to_drop)} redundant columns. New shape: {df.shape}")

# 2️⃣ Replace region codes (REG or AGGLOIFOP0)
region_map = {
    "UDA1": "Île-de-France",
    "UDA2": "Hauts-de-France",
    "UDA3": "Grand Est",
    "UDA4": "Bourgogne-Franche-Comté",
    "UDA5": "Auvergne-Rhône-Alpes",
    "UDA6": "Provence-Alpes-Côte d’Azur",
    "UDA7": "Occitanie",
    "UDA8": "Nouvelle-Aquitaine",
    "UDA9": "Pays de la Loire",
}

urban_map = {
    "UU01": "Paris et grandes métropoles",
    "UU02": "Grande ville (100k–500k hab.)",
    "UU03": "Ville moyenne (50k–100k hab.)",
    "UU04": "Petite ville (20k–50k hab.)",
    "UU05": "Bourg / petite agglomération",
    "UU06": "Rural périurbain",
    "UU07": "Rural isolé",
    "UU08": "Autres / hors unité urbaine",
    "Hors unité urbaine": "Rural isolé"
}

if "REG" in df.columns:
    df["REG"] = df["REG"].replace(region_map)

if "AGGLOIFOP0" in df.columns:
    df["AGGLOIFOP0"] = df["AGGLOIFOP0"].replace(urban_map)

# 3️⃣ Replace PPIA codes (Profession principale)
ppia_map = {
    1: "Agriculteurs exploitants",
    2: "Artisans (moins de 10 salariés)",
    3: "Commerçants et assimilés (moins de 10 salariés)",
    4: "Chefs d'entreprise de 10 salariés ou plus",
    5: "Professions libérales de la santé",
    6: "Cadres de la fonction publique",
    7: "Professeurs, professions scientifiques",
    8: "Professions de l'information, arts et spectacles",
    9: "Cadres d'entreprise (administratifs et commerciaux)",
    10: "Ingénieurs et cadres techniques d'entreprise",
    11: "Professeurs des écoles, instituteurs",
    12: "Professions intermédiaires de la santé et du travail social",
    13: "Clergé, religieux",
    14: "Professions intermédiaires administratives de la fonction publique",
    15: "Professions intermédiaires commerciales et administratives",
    16: "Techniciens (géomètres, dessinateurs, etc.)",
    17: "Contremaîtres, agents de maîtrise",
    18: "Employés civils et agents de service publics",
    19: "Policiers, militaires et gendarmes",
    20: "Employés administratifs d'entreprise",
    21: "Employés de commerce",
    22: "Services directs aux particuliers",
    23: "Ouvriers qualifiés de type industriel",
    24: "Ouvriers qualifiés de type artisanal",
    25: "Chauffeurs",
    26: "Ouvriers de manutention et du transport",
    27: "Ouvriers non qualifiés industriels",
    28: "Ouvriers non qualifiés artisanaux",
    29: "Ouvriers agricoles et assimilés",
    30: "Vous n'avez jamais travaillé"
}

if "PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d" in df.columns:
    col = "PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d"
    df[col] = df[col].replace(ppia_map)

# 4️⃣ Replace STC codes (Statut professionnel)
stc_map = {
    1: "Chef d'entreprise à son compte",
    2: "Chef d’entreprise salarié / gérant / PDG",
    3: "Salarié du privé ou association",
    4: "Salarié d’entreprise publique",
    5: "Salarié de l’État",
    6: "Salarié collectivité territoriale / HLM",
    7: "Salarié d’un hôpital public",
    8: "Salarié de la Sécurité sociale",
    9: "Salarié d’un particulier",
    10: "Travaille pour un membre de la famille non rémunéré"
}

if "STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal." in df.columns:
    col = "STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal."
    df[col] = df[col].replace(stc_map)

# 5️⃣ Save cleaned dataset
df.to_csv("barometre_final_labeled.csv", index=False, encoding="utf-8")
print(f"💾 Cleaned dataset saved: barometre_final_labeled.csv")
print(f"✅ Final shape: {df.shape}")

# Preview final replacements
display(df.head(3))


Initial shape: (5041, 40)
🧹 Dropped 2 redundant columns. New shape: (5041, 38)
💾 Cleaned dataset saved: barometre_final_labeled.csv
✅ Final shape: (5041, 38)


,SEXE - Vous êtes... ?,AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :,AGGLOIFOP0,TYPCOM,TAILCOM,DPT,REG,"SITI - Actuellement, quelle est votre situation ?","PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d",RECPPIA,"STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal.",STCA,"STATUT - Au sein de votre foyer, quelle est votre situation ?",FOYER - De combien de personnes se compose votre foyer y compris vous-même ?,"ENF - Au total, combien y a-t-il d’enfants de moins de 18 ans dans votre foyer ?",- Quel est l'âge de votre / vos enfant(s) vivant dans votre foyer ? Vous pouvez sélectionner plusieurs réponses._1,"RS6 - A quelle fréquence utilisez-vous Internet ou des applications, quels que soient le lieu d’utilisation et l’appareil de connexion ?",Q2 - À quelle fréquence consommez-vous sur Internet chacun des produits ou services culturels dématérialisés suivants ? Une réponse par ligne. Vous consommez sur Internet …_r1,"Q5 - Plus précisément, pour chacun des produits ou services culturels suivants, diriez-vous que vous les consommez… Une réponse par ligne. sur Internet …_r1","Q7 - Concernant votre consommation de biens culturels dématérialisés, diriez-vous qu’aujourd’hui :",QBU1 - Vous nous avez dit consommer de façon dématérialisée les contenus culturels et sportifs suivants. Veuillez indiquer pour chacun d’eux si vous les consommez gratuitement ou de façon payante. On parle toujours de contenus culturels et spor_r1,AFFi1,"QBU2 - De façon générale, quel montant dépensez-vous en moyenne chaque mois pour votre consommation de [% ListLabel(Q1List,AFFi1) %] [% ListLabel(Q1List,AFFi2) %] [% ListLabel(Q1List,AFFi3) %] [% ListLabel(Q1List,AFFi4) %] [% ListLabel(Q1List,AFFi5","- Utilisez-vous des applications « crackées » que vous avez téléchargées sur des stores d’applications alternatifs (comme AppValley ou Tutuapp par exemple) ou via des APKs, permettant l’accès à des offres payantes sans payer ? Vou_1","QBU12 - Utilisez-vous des logiciels, des applications ou des sites internet permettant de convertir des contenus consultés en streaming (films, séries, musique vus sur une plateforme) en un contenu à télécharger (qui permettent par exemple de conv_r1","- Parmi les équipements suivants, le(s)quel(s) possédez-vous à titre personnel ? Vous pouvez sélectionner plusieurs réponses._1",RS8 - Et vous arrive-t-il de faire des réglages de DNS ?,- A quelle(s) chaîne(s) payante(s) avez-vous accès ? Vous pouvez sélectionner plusieurs réponses._1,- A quels services ou applications de musique payants avez-vous accès (y compris de manière associée à un forfait téléphonique) ? Vous pouvez sélectionner plusieurs réponses._1,- A quel(s) service(s) de vidéo à la demande par abonnement payant avez-vous accès? Vous pouvez sélectionner plusieurs réponses._1,RS15BIS - Certains services de vidéo à la demande par abonnement proposent désormais des formules d’abonnement moins chères mais incluant de la publicité. Seriez-vous prêt à souscrire à une offre avec publicité pour réduire le coût de l’,DISPQUAL,"RS7BIS - Au cours des 12 derniers mois, avez-vous utilisé au moins un VPN à titre personnel ?","QBU5a - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","QBU5b - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","RS12BIS - Avez-vous accès aux fournisseurs de services payants suivants ? Attention, nous parlons ici des offres auxquelles vous avez accè

In [95]:
df.to_excel("final.xlsx", index=False, engine='openpyxl')

In [1]:


# Load your latest dataset
df = pd.read_excel("final.xlsx")

print(f"Initial shape: {df.shape}")

# ====================================================
# 1️⃣ Drop technical / non-analytic columns
# ====================================================
cols_to_drop = ["AFFi1", "DISPQUAL"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
print(f"🧹 Dropped {len(cols_to_drop)} technical columns. New shape: {df.shape}")

# ====================================================
# 2️⃣ Define mapping dictionaries for key questions
# ====================================================

# -- Age of children
age_children_map = {
    1: "1 an ou moins", 2: "2 ans", 3: "3 ans", 4: "4 ans", 5: "5 ans",
    6: "6 ans", 7: "7 ans", 8: "8 ans", 9: "9 ans", 10: "10 ans",
    11: "11 ans", 12: "12 ans", 13: "13 ans", 14: "14 ans",
    15: "15 ans", 16: "16 ans", 17: "17 ans"
}

# -- Frequency of consumption (Q2)
q2_map = {
    1: "Tous les jours ou presque",
    2: "1 à 5 fois par semaine",
    3: "1 à 3 fois par mois",
    4: "Moins souvent"
}

# -- Legal vs illegal consumption (Q5)
q5_map = {
    1: "Uniquement de façon légale",
    2: "Généralement légale, parfois illégale",
    3: "Autant légale qu’illégale",
    4: "Généralement illégale, parfois légale",
    5: "Uniquement de manière illégale"
}

# -- Change over time (Q7)
q7_map = {
    1: "Consomme davantage de façon légale qu’avant",
    2: "Consomme autant de façon légale qu’avant",
    3: "Consomme moins de façon légale qu’avant",
    4: "Ne consomme pas du tout de manière légale"
}

# ====================================================
# 3️⃣ Apply mappings if columns exist
# ====================================================
def safe_replace(df, colname, mapping):
    for c in df.columns:
        if colname.lower() in c.lower():
            df[c] = df[c].replace(mapping)
            print(f"✅ Replaced codes in: {c}")
    return df

df = safe_replace(df, "Quel est l'âge de votre / vos enfant(s)", age_children_map)
df = safe_replace(df, "Q2 - À quelle fréquence consommez-vous", q2_map)
df = safe_replace(df, "Q5 - Plus précisément, pour chacun", q5_map)
df = safe_replace(df, "Q7 - Concernant votre consommation", q7_map)

# ====================================================
# 4️⃣ Save cleaned dataset
# ====================================================
df.to_csv("barometre_final_ready.csv", index=False, encoding="utf-8")
print(f"💾 Cleaned dataset saved: barometre_final_ready.csv")
print(f"✅ Final shape: {df.shape}")

# Quick verification of changes
for pattern in ["Q2", "Q5", "Q7", "enfant"]:
    cols = [c for c in df.columns if pattern in c]
    if cols:
        print(f"\n🔍 Sample values for {pattern}:")
        for c in cols:
            print(f" - {c}: {df[c].dropna().unique()[:5]}")


Initial shape: (5041, 38)
🧹 Dropped 2 technical columns. New shape: (5041, 36)
✅ Replaced codes in: - Quel est l'âge de votre / vos enfant(s) vivant dans votre foyer ? Vous pouvez sélectionner plusieurs réponses._1
✅ Replaced codes in: Q2 - À quelle fréquence consommez-vous sur Internet chacun des produits ou services culturels dématérialisés suivants ? Une réponse par ligne. Vous consommez sur Internet …_r1
✅ Replaced codes in: Q5 - Plus précisément, pour chacun des produits ou services culturels suivants, diriez-vous que vous les consommez… Une réponse par ligne. sur Internet …_r1
✅ Replaced codes in: Q7 - Concernant votre consommation de biens culturels dématérialisés, diriez-vous qu’aujourd’hui :
💾 Cleaned dataset saved: barometre_final_ready.csv
✅ Final shape: (5041, 36)

🔍 Sample values for Q2:
 - Q2 - À quelle fréquence consommez-vous sur Internet chacun des produits ou services culturels dématérialisés suivants ? Une réponse par ligne. Vous consommez sur Internet …_r1: ['1 à 5 

In [5]:
df.to_excel("final.xlsx", index=False, engine='openpyxl')
df = pd.read_excel(path_data)

NameError: name 'df' is not defined

In [7]:


# Load the current dataset
df = pd.read_csv("barometre_final_ready.csv")
print(f"Initial shape: {df.shape}")

# ====================================================
# 1️⃣ Remove irrelevant or redundant columns
# ====================================================

cols_to_drop = [
    "- Quel est l'âge de votre / vos enfant(s) vivant dans votre foyer ? Vous pouvez sélectionner plusieurs réponses._1",
    "- Parmi les équipements suivants, le(s)quel(s) possédez-vous à titre personnel ? Vous pouvez sélectionner plusieurs réponses._1",
    "- A quelle(s) chaîne(s) payante(s) avez-vous accès ? Vous pouvez sélectionner plusieurs réponses._1",
    "- A quels services ou applications de musique payants avez-vous accès (y compris de manière associée à un forfait téléphonique) ? Vous pouvez sélectionner plusieurs réponses._1",
    "- A quel(s) service(s) de vidéo à la demande par abonnement payant avez-vous accès? Vous pouvez sélectionner plusieurs réponses._1",
    "RS15BIS - Certains services de vidéo à la demande par abonnement proposent désormais des formules d’abonnement moins chères mais incluant de la publicité. Seriez-vous prêt à souscrire à une offre avec publicité pour réduire le coût de l’"
]

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
print(f"🧹 Dropped {len(cols_to_drop)} columns. New shape: {df.shape}")

# ====================================================
# 2️⃣ Mapping dictionaries from your screenshots
# ====================================================

# QBU1
qbu1_map = {
    1: "Uniquement gratuitement",
    2: "Le plus souvent gratuitement, mais parfois de façon payante",
    3: "Autant gratuitement que de façon payante",
    4: "Le plus souvent de façon payante, mais parfois gratuitement",
    5: "Uniquement de façon payante"
}

# Applis crackées
apps_crack_map = {
    1: "Des applis crackées permettant l’accès gratuit à une offre payante de chaînes de télévision (Canal+, beIN, etc.)",
    2: "Des applis crackées permettant l’accès gratuit à une offre payante de vidéo à la demande (Netflix, Canal+ Séries, etc.)",
    3: "Des applis crackées permettant l’accès gratuit à une offre payante de musique (Deezer, Apple Music, Spotify, etc.)",
    4: "Non, je n’utilise pas d’application crackée",
    5: "Je ne sais pas de quoi il s’agit"
}

# QBU12
qbu12_map = {
    1: "Tous les jours ou presque",
    2: "1 à 5 fois par semaine",
    3: "1 à 3 fois par mois",
    4: "Moins souvent",
    5: "Jamais"
}

# ====================================================
# 3️⃣ Apply mappings
# ====================================================

def safe_replace(df, col_pattern, mapping):
    """Replace values in all columns that match part of a name."""
    matched = [c for c in df.columns if col_pattern.lower() in c.lower()]
    for col in matched:
        df[col] = df[col].replace(mapping)
        print(f"✅ Replaced codes in: {col}")
    if not matched:
        print(f"⚠️ No column found for pattern '{col_pattern}'")
    return df

df = safe_replace(df, "QBU1", qbu1_map)
df = safe_replace(df, "crack", apps_crack_map)
df = safe_replace(df, "QBU12", qbu12_map)

# ====================================================
# 4️⃣ Save final cleaned dataset
# ====================================================

df.to_csv("barometre_final_clean_labeled.csv", index=False, encoding="utf-8")
print(f"💾 Clean labeled dataset saved: barometre_final_clean_labeled.csv")
print(f"✅ Final shape: {df.shape}")

# Quick check of mapping results
for pattern in ["QBU1", "crack", "QBU12"]:
    cols = [c for c in df.columns if pattern.lower() in c.lower()]
    if cols:
        print(f"\n🔍 Sample values for '{pattern}':")
        for c in cols:
            print(f" - {c}: {df[c].dropna().unique()[:5]}")

df.to_excel("final.xlsx", index=False, engine='openpyxl')

Initial shape: (5041, 36)
🧹 Dropped 6 columns. New shape: (5041, 30)
✅ Replaced codes in: QBU1 - Vous nous avez dit consommer de façon dématérialisée les contenus culturels et sportifs suivants. Veuillez indiquer pour chacun d’eux si vous les consommez gratuitement ou de façon payante. On parle toujours de contenus culturels et spor_r1
✅ Replaced codes in: QBU12 - Utilisez-vous des logiciels, des applications ou des sites internet permettant de convertir des contenus consultés en streaming (films, séries, musique vus sur une plateforme) en un contenu à télécharger (qui permettent par exemple de conv_r1
✅ Replaced codes in: - Utilisez-vous des applications « crackées » que vous avez téléchargées sur des stores d’applications alternatifs (comme AppValley ou Tutuapp par exemple) ou via des APKs, permettant l’accès à des offres payantes sans payer ? Vou_1
✅ Replaced codes in: QBU12 - Utilisez-vous des logiciels, des applications ou des sites internet permettant de convertir des contenus co

In [9]:


print(f"Initial shape: {df.shape}")

# ====================================================
# 1️⃣ Supprimer la colonne RS17A
# ====================================================
col_to_drop = [
    "RS17A - Quel budget consacrez-vous au sein de votre foyer à ces types d’abonnements ? Vous pouvez indiquer un budget mensuel ou annuel. Saisissez vos réponses dans les cadres ci-dessous. Une seule réponse par ligne. € / mois Vos abonnements à_COMBINED"
]
df = df.drop(columns=[c for c in col_to_drop if c in df.columns])
print(f"🧹 Dropped RS17A column. New shape: {df.shape}")

# ====================================================
# 2️⃣ Dictionnaires de correspondance (tes captures)
# ====================================================
qbu5a_map = {
    0: None,
    1: "Sur un ordinateur",
    2: "Sur un smartphone",
    3: "Sur une tablette",
    4: "Sur un écran de télévision",
    5: "Sur une console de jeu portable (Nintendo Switch, DS, PS Vita)",
    6: "Sur d’autres appareils"
}

qbu5b_map = {
    0: None,
    1: "Sur un ordinateur",
    2: "Sur un smartphone",
    3: "Sur une tablette",
    4: "Sur d’autres appareils"
}

rs12bis_map = {
    0: None,
    1: "J’y suis abonné au sein de mon foyer",
    2: "Je bénéficie d’un compte d’une personne extérieure à mon foyer",
    3: "Je n’ai pas accès à ce type d’abonnement"
}

rs16bis_map = {
    0: None,
    1: "Des membres de ma famille",
    2: "Des amis",
    3: "Des codes trouvés gratuitement sur Internet",
    4: "D’autres personnes"
}

# ====================================================
# 3️⃣ Fonction de transformation : "code 1, code 3" → "Texte 1, Texte 3"
# ====================================================
def decode_codes(cell, mapping):
    if pd.isna(cell):
        return None
    codes = re.findall(r"\d+", str(cell))  # extrait 1, 3, 5...
    if not codes:
        return None
    decoded = [mapping.get(int(c), f"Inconnu ({c})") for c in codes if int(c) in mapping]
    return ", ".join([d for d in decoded if d])

# ====================================================
# 4️⃣ Appliquer le décodage sur les 4 colonnes concernées
# ====================================================
def decode_column(df, pattern, mapping):
    matched = [c for c in df.columns if pattern.lower() in c.lower()]
    for col in matched:
        df[col] = df[col].apply(lambda x: decode_codes(x, mapping))
        print(f"✅ Decoded codes in: {col}")
    if not matched:
        print(f"⚠️ No match found for pattern '{pattern}'")
    return df

df = decode_column(df, "QBU5a", qbu5a_map)
df = decode_column(df, "QBU5b", qbu5b_map)
df = decode_column(df, "RS12BIS", rs12bis_map)
df = decode_column(df, "RS16BIS", rs16bis_map)

# ====================================================
# 5️⃣ Remplacer 0 par "Non" pour la question crackée
# ====================================================
crack_col = [c for c in df.columns if "crack" in c.lower()]
for col in crack_col:
    df[col] = df[col].replace(0, "Non")
    print(f"✅ Replaced 0 → 'Non' in {col}")

# ====================================================
# 6️⃣ Sauvegarde
# ====================================================
df.to_csv("barometre_final_decoded.csv", index=False, encoding="utf-8")
print(f"💾 Saved decoded dataset: barometre_final_decoded.csv")
print(f"✅ Final shape: {df.shape}")

# Vérification rapide
for pattern in ["QBU5a", "QBU5b", "RS12BIS", "RS16BIS"]:
    cols = [c for c in df.columns if pattern.lower() in c.lower()]
    if cols:
        print(f"\n🔍 Sample values for {pattern}:")
        for c in cols:
            print(f" - {c}: {df[c].dropna().unique()[:5]}")

df.to_excel("final.xlsx", index=False, engine='openpyxl')

Initial shape: (5041, 30)
🧹 Dropped RS17A column. New shape: (5041, 29)
✅ Decoded codes in: QBU5a - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED
✅ Decoded codes in: QBU5b - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED
✅ Decoded codes in: RS12BIS - Avez-vous accès aux fournisseurs de services payants suivants ? Attention, nous parlons ici des offres auxquelles vous avez accès en payant (vous ou une autre personne de votre foyer) ou en bénéficiant d’un compte d’une personne ext_COMBINED
✅ Decoded codes in: RS16BIS - À qui appartiennent ces codes d’accès extérieurs à votre foyer que vous utilisez ? Vous pouvez sélectionner plusieurs réponses par ligne. Pour_COMBINE

In [13]:
df.head()

,SEXE - Vous êtes... ?,AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :,AGGLOIFOP0,TYPCOM,TAILCOM,DPT,REG,"SITI - Actuellement, quelle est votre situation ?","PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d",RECPPIA,...,QBU1 - Vous nous avez dit consommer de façon dématérialisée les contenus culturels et sportifs suivants. Veuillez indiquer pour chacun d’eux si vous les consommez gratuitement ou de façon payante. On parle toujours de contenus culturels et spor_r1,"QBU2 - De façon générale, quel montant dépensez-vous en moyenne chaque mois pour votre consommation de [% ListLabel(Q1List,AFFi1) %] [% ListLabel(Q1List,AFFi2) %] [% ListLabel(Q1List,AFFi3) %] [% ListLabel(Q1List,AFFi4) %] [% ListLabel(Q1List,AFFi5","- Utilisez-vous des applications « crackées » que vous avez téléchargées sur des stores d’applications alternatifs (comme AppValley ou Tutuapp par exemple) ou via des APKs, permettant l’accès à des offres payantes sans payer ? Vou_1","QBU12 - Utilisez-vous des logiciels, des applications ou des sites internet permettant de convertir des contenus consultés en streaming (films, séries, musique vus sur une plateforme) en un contenu à télécharger (qui permettent par exemple de conv_r1",RS8 - Et vous arrive-t-il de faire des réglages de DNS ?,"RS7BIS - Au cours des 12 derniers mois, avez-vous utilisé au moins un VPN à titre personnel ?","QBU5a - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","QBU5b - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","RS12BIS - Avez-vous accès aux fournisseurs de services payants suivants ? Attention, nous parlons ici des offres auxquelles vous avez accès en payant (vous ou une autre personne de votre foyer) ou en bénéficiant d’un compte d’une personne ext_COMBINED",RS16BIS - À qui appartiennent ces codes d’accès extérieurs à votre foyer que vous utilisez ? Vous pouvez sélectionner plusieurs réponses par ligne. Pour_COMBINED
0,Un homme,52,Grande ville (100k–500k hab.),Rural,De3500a4999hab,OISE,Bourgogne-Franche-Comté,"Vous êtes dans une autre situation (invalide, ...",NaN,Autres1,...,NaN,NaN,NaN,NaN,Jamais,"Non, vous n’avez jamais utilisé de VPN à titre...",None,None,Je bénéficie d’un compte d’une personne extéri...,Des membres de ma famille
1,Un homme,56,Autres / hors unité urbaine,BanlieueNVSup,De100000habEtPlus,HAUTSDESEINE,Île-de-France,Vous exercez une activité professionnelle (act...,Ingénieurs et cadres techniques d'entreprise,Cadres1,...,Uniquement gratuitement,0.0,Non,Autant gratuitement que de façon payante,Jamais,"Non, vous n’avez jamais utilisé de VPN à titre...","Sur un ordinateur, Sur un smartphone",Sur d’autres appareils,,None
2,Une femme,40,Autres / hors unité urbaine,BanlieueNVModeste,De50000a99999hab,SEINESAINTDENIS,Île-de-France,Vous exercez une activité professionnelle (act...,Employés administratifs d'entreprise,Employes1,...,NaN,30.0,Des applis crackées permettant l’accès gratuit...,Uniquement gratuitement,Jamais,"Oui, vous avez occasionnellement utilisé un VP...","Sur un smartphone, Sur une tablette, Sur une c...",Sur d’autres appareils,J’y suis abonné au sein de mon foyer,"Des amis, Des codes trouvés gratuitement sur I..."
3,Un homme,49,Grande ville (100k–500k hab.),VilleIsolee,De5000a8999hab,YONNE,Bourgogne-Franche-Comté,Vous exercez une activité professionnelle (act...,"Professeurs, professions scientifiques",Cadres1,...,NaN,25.0,Non,Autant gratuitement que de façon payante,De temps en temps,"Oui, vous avez occasionnellement

In [15]:
list(df.columns)


['SEXE - Vous êtes... ?',
 'AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :',
 'AGGLOIFOP0',
 'TYPCOM',
 'TAILCOM',
 'DPT',
 'REG',
 'SITI - Actuellement, quelle est votre situation ?',
 'PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d',
 'RECPPIA',
 'STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal.',
 'STCA',
 'STATUT - Au sein de votre foyer, quelle est votre situation ?',
 'FOYER - De combien de personnes se compose votre foyer y compris vous-même ?',
 'ENF - Au total, combien y a-t-il d’enfants de moins de 18 ans dans votre foyer ?',
 'RS6 - A quelle fréquence utilisez-vous Internet ou des applications, quels que soient le lieu d’utilisation et l’appareil de connexion ?',
 'Q2 - À q

In [33]:
# ============================================================
# 1️⃣ Imports & dataset
# ============================================================

plt.style.use("seaborn-v0_8-whitegrid")

df = pd.read_excel("final.xlsx")

print(f"✅ Dataset loaded: {df.shape[0]} rows × {df.shape[1]} columns")
df.head(3)


✅ Dataset loaded: 5041 rows × 29 columns


,SEXE - Vous êtes... ?,AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :,AGGLOIFOP0,TYPCOM,TAILCOM,DPT,REG,"SITI - Actuellement, quelle est votre situation ?","PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d",RECPPIA,...,QBU1 - Vous nous avez dit consommer de façon dématérialisée les contenus culturels et sportifs suivants. Veuillez indiquer pour chacun d’eux si vous les consommez gratuitement ou de façon payante. On parle toujours de contenus culturels et spor_r1,"QBU2 - De façon générale, quel montant dépensez-vous en moyenne chaque mois pour votre consommation de [% ListLabel(Q1List,AFFi1) %] [% ListLabel(Q1List,AFFi2) %] [% ListLabel(Q1List,AFFi3) %] [% ListLabel(Q1List,AFFi4) %] [% ListLabel(Q1List,AFFi5","- Utilisez-vous des applications « crackées » que vous avez téléchargées sur des stores d’applications alternatifs (comme AppValley ou Tutuapp par exemple) ou via des APKs, permettant l’accès à des offres payantes sans payer ? Vou_1","QBU12 - Utilisez-vous des logiciels, des applications ou des sites internet permettant de convertir des contenus consultés en streaming (films, séries, musique vus sur une plateforme) en un contenu à télécharger (qui permettent par exemple de conv_r1",RS8 - Et vous arrive-t-il de faire des réglages de DNS ?,"RS7BIS - Au cours des 12 derniers mois, avez-vous utilisé au moins un VPN à titre personnel ?","QBU5a - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","QBU5b - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED","RS12BIS - Avez-vous accès aux fournisseurs de services payants suivants ? Attention, nous parlons ici des offres auxquelles vous avez accès en payant (vous ou une autre personne de votre foyer) ou en bénéficiant d’un compte d’une personne ext_COMBINED",RS16BIS - À qui appartiennent ces codes d’accès extérieurs à votre foyer que vous utilisez ? Vous pouvez sélectionner plusieurs réponses par ligne. Pour_COMBINED
0,Un homme,52,Grande ville (100k–500k hab.),Rural,De3500a4999hab,OISE,Bourgogne-Franche-Comté,"Vous êtes dans une autre situation (invalide, ...",NaN,Autres1,...,NaN,NaN,NaN,NaN,Jamais,"Non, vous n’avez jamais utilisé de VPN à titre...",NaN,NaN,Je bénéficie d’un compte d’une personne extéri...,Des membres de ma famille
1,Un homme,56,Autres / hors unité urbaine,BanlieueNVSup,De100000habEtPlus,HAUTSDESEINE,Île-de-France,Vous exercez une activité professionnelle (act...,Ingénieurs et cadres techniques d'entreprise,Cadres1,...,Uniquement gratuitement,0.0,Non,Autant gratuitement que de façon payante,Jamais,"Non, vous n’avez jamais utilisé de VPN à titre...","Sur un ordinateur, Sur un smartphone",Sur d’autres appareils,NaN,NaN
2,Une femme,40,Autres / hors unité urbaine,BanlieueNVModeste,De50000a99999hab,SEINESAINTDENIS,Île-de-France,Vous exercez une activité professionnelle (act...,Employés administratifs d'entreprise,Employes1,...,NaN,30.0,Des applis crackées permettant l’accès gratuit...,Uniquement gratuitement,Jamais,"Oui, vous avez occasionnellement utilisé un VP...","Sur un smartphone, Sur une tablette, Sur une c...",Sur d’autres appareils,J’y suis abonné au sein de mon foyer,"Des amis, Des codes trouvés gratuitement sur I..."


In [37]:
rename_dict = {
    "SEXE - Vous êtes... ?": "sexe",
    "AGE - Quel âge avez-vous ? Merci de noter votre âge dans le cadre ci-dessous :": "age",
    "AGGLOIFOP0": "type_agglomeration",
    "TYPCOM": "type_commune",
    "TAILCOM": "taille_commune",
    "DPT": "departement",
    "REG": "region",
    "SITI - Actuellement, quelle est votre situation ?": "situation_personnelle",
    "PPIA - Plus précisément, quelle est votre profession principale ou, si vous ne travaillez pas actuellement, la dernière profession principale que vous avez exercée ? Attention, si vous n’avez fait dans votre vie que des petits boulots (ex : job d": "profession_principale",
    "RECPPIA": "statut_professionnel",
    "STC - Vous exercez cette profession comme… ? Si vous exercez plusieurs emplois, décrivez uniquement votre emploi principal.": "statut_emploi",
    "STCA": "categorie_socio_professionnelle",
    "STATUT - Au sein de votre foyer, quelle est votre situation ?": "statut_foyer",
    "FOYER - De combien de personnes se compose votre foyer y compris vous-même ?": "taille_foyer",
    "ENF - Au total, combien y a-t-il d’enfants de moins de 18 ans dans votre foyer ?": "nb_enfants",
    "RS6 - A quelle fréquence utilisez-vous Internet ou des applications, quels que soient le lieu d’utilisation et l’appareil de connexion ?": "frequence_internet",
    "Q2 - À quelle fréquence consommez-vous sur Internet chacun des produits ou services culturels dématérialisés suivants ? Une réponse par ligne. Vous consommez sur Internet …_r1": "frequence_conso_culturelle",
    "Q5 - Plus précisément, pour chacun des produits ou services culturels suivants, diriez-vous que vous les consommez… Une réponse par ligne. sur Internet …_r1": "type_conso_legale_ou_illegale",
    "Q7 - Concernant votre consommation de biens culturels dématérialisés, diriez-vous qu’aujourd’hui :": "evolution_conso_legale",
    "QBU1 - Vous nous avez dit consommer de façon dématérialisée les contenus culturels et sportifs suivants. Veuillez indiquer pour chacun d’eux si vous les consommez gratuitement ou de façon payante. On parle toujours de contenus culturels et spor_r1": "gratuit_ou_payant",
    "QBU2 - De façon générale, quel montant dépensez-vous en moyenne chaque mois pour votre consommation de [% ListLabel(Q1List,AFFi1) %] [% ListLabel(Q1List,AFFi2) %] [% ListLabel(Q1List,AFFi3) %] [% ListLabel(Q1List,AFFi4) %] [% ListLabel(Q1List,AFFi5": "depense_mensuelle_culturelle",
    "- Utilisez-vous des applications « crackées » que vous avez téléchargées sur des stores d’applications alternatifs (comme AppValley ou Tutuapp par exemple) ou via des APKs, permettant l’accès à des offres payantes sans payer ? Vou_1": "utilisation_applis_crackees",
    "QBU12 - Utilisez-vous des logiciels, des applications ou des sites internet permettant de convertir des contenus consultés en streaming (films, séries, musique vus sur une plateforme) en un contenu à télécharger (qui permettent par exemple de conv_r1": "utilisation_telechargement_streaming",
    "RS8 - Et vous arrive-t-il de faire des réglages de DNS ?": "reglages_dns",
    "RS7BIS - Au cours des 12 derniers mois, avez-vous utilisé au moins un VPN à titre personnel ?": "utilisation_vpn",
    "QBU5a - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED": "appareils_conso_musique_videos",
    "QBU5b - Et au cours des 12 derniers mois, sur quels appareils avez-vous consommé ces contenus culturels et sportifs la plupart du temps ? Vous pouvez sélectionner plusieurs réponses par ligne. Généralement, …_COMBINED": "appareils_conso_films_series",
    "RS12BIS - Avez-vous accès aux fournisseurs de services payants suivants ? Attention, nous parlons ici des offres auxquelles vous avez accès en payant (vous ou une autre personne de votre foyer) ou en bénéficiant d’un compte d’une personne ext_COMBINED": "acces_services_payants",
    "RS16BIS - À qui appartiennent ces codes d’accès extérieurs à votre foyer que vous utilisez ? Vous pouvez sélectionner plusieurs réponses par ligne. Pour_COMBINED": "provenance_codes_acces_exterieurs"
}

# Application du renommage
df = df.rename(columns=rename_dict)
print("✅ Colonnes renommées avec succès !")
print(df.columns.tolist())
df.to_csv("barometre_final_decoded.csv", index=False, encoding="utf-8")

✅ Colonnes renommées avec succès !
['sexe', 'age', 'type_agglomeration', 'type_commune', 'taille_commune', 'departement', 'region', 'situation_personnelle', 'profession_principale', 'statut_professionnel', 'statut_emploi', 'categorie_socio_professionnelle', 'statut_foyer', 'taille_foyer', 'nb_enfants', 'frequence_internet', 'frequence_conso_culturelle', 'type_conso_legale_ou_illegale', 'evolution_conso_legale', 'gratuit_ou_payant', 'depense_mensuelle_culturelle', 'utilisation_applis_crackees', 'utilisation_telechargement_streaming', 'reglages_dns', 'utilisation_vpn', 'appareils_conso_musique_videos', 'appareils_conso_films_series', 'acces_services_payants', 'provenance_codes_acces_exterieurs']


In [39]:
df.head()

,sexe,age,type_agglomeration,type_commune,taille_commune,departement,region,situation_personnelle,profession_principale,statut_professionnel,...,gratuit_ou_payant,depense_mensuelle_culturelle,utilisation_applis_crackees,utilisation_telechargement_streaming,reglages_dns,utilisation_vpn,appareils_conso_musique_videos,appareils_conso_films_series,acces_services_payants,provenance_codes_acces_exterieurs
0,Un homme,52,Grande ville (100k–500k hab.),Rural,De3500a4999hab,OISE,Bourgogne-Franche-Comté,"Vous êtes dans une autre situation (invalide, ...",NaN,Autres1,...,NaN,NaN,NaN,NaN,Jamais,"Non, vous n’avez jamais utilisé de VPN à titre...",NaN,NaN,Je bénéficie d’un compte d’une personne extéri...,Des membres de ma famille
1,Un homme,56,Autres / hors unité urbaine,BanlieueNVSup,De100000habEtPlus,HAUTSDESEINE,Île-de-France,Vous exercez une activité professionnelle (act...,Ingénieurs et cadres techniques d'entreprise,Cadres1,...,Uniquement gratuitement,0.0,Non,Autant gratuitement que de façon payante,Jamais,"Non, vous n’avez jamais utilisé de VPN à titre...","Sur un ordinateur, Sur un smartphone",Sur d’autres appareils,NaN,NaN
2,Une femme,40,Autres / hors unité urbaine,BanlieueNVModeste,De50000a99999hab,SEINESAINTDENIS,Île-de-France,Vous exercez une activité professionnelle (act...,Employés administratifs d'entreprise,Employes1,...,NaN,30.0,Des applis crackées permettant l’accès gratuit...,Uniquement gratuitement,Jamais,"Oui, vous avez occasionnellement utilisé un VP...","Sur un smartphone, Sur une tablette, Sur une c...",Sur d’autres appareils,J’y suis abonné au sein de mon foyer,"Des amis, Des codes trouvés gratuitement sur I..."
3,Un homme,49,Grande ville (100k–500k hab.),VilleIsolee,De5000a8999hab,YONNE,Bourgogne-Franche-Comté,Vous exercez une activité professionnelle (act...,"Professeurs, professions scientifiques",Cadres1,...,NaN,25.0,Non,Autant gratuitement que de façon payante,De temps en temps,"Oui, vous avez occasionnellement utilisé un VP...","Sur un smartphone, Sur une tablette, Sur une c...",NaN,J’y suis abonné au sein de mon foyer,Des amis
4,Une femme,50,Rural isolé,VilleCentre,De100000habEtPlus,MAINEETLOIRE,Provence-Alpes-Côte d’Azur,Vous exercez une activité professionnelle (act...,Employés administratifs d'entreprise,Employes1,...,NaN,0.0,Non,Uniquement de façon payante,Je ne sais pas ce que c’est,"Oui, vous avez régulièrement utilisé un VPN à ...",NaN,Sur d’autres appareils,NaN,NaN


In [11]:
df = pd.read_csv("barometre_final_decoded.csv")

In [13]:
# ============================================================
# 2️⃣ Sélectionner les colonnes pertinentes
# ============================================================
cols_keep = [
    "sexe", "age", "region", "type_agglomeration",
    "situation_personnelle", "profession_principale", "statut_emploi",
    "frequence_internet", "frequence_conso_culturelle",
    "type_conso_legale_ou_illegale", "evolution_conso_legale",
    "gratuit_ou_payant", "depense_mensuelle_culturelle",
    "appareils_conso_musique_videos", "appareils_conso_films_series",
    "utilisation_vpn", "utilisation_applis_crackees",
    "utilisation_telechargement_streaming", "reglages_dns",
    "acces_services_payants", "provenance_codes_acces_exterieurs",
    "taille_foyer", "nb_enfants", "statut_foyer"
]

df = df[[c for c in cols_keep if c in df.columns]].copy()

print(f"✅ Columns kept: {len(df.columns)}")
print(df.info())


✅ Columns kept: 24
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5041 entries, 0 to 5040
Data columns (total 24 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   sexe                                  5041 non-null   object 
 1   age                                   5041 non-null   int64  
 2   region                                5041 non-null   object 
 3   type_agglomeration                    5041 non-null   object 
 4   situation_personnelle                 5041 non-null   object 
 5   profession_principale                 4223 non-null   object 
 6   statut_emploi                         3179 non-null   object 
 7   frequence_internet                    5041 non-null   object 
 8   frequence_conso_culturelle            2903 non-null   object 
 9   type_conso_legale_ou_illegale         2903 non-null   object 
 10  evolution_conso_legale                4599 non-null   object 
 11

In [15]:
# ============================================================
# 3️⃣ Qualité des données
# ============================================================
print("\n📊 Missing values (%):")
print((df.isna().mean() * 100).round(1).sort_values(ascending=False))

print("\n🔁 Duplicate rows:", df.duplicated().sum())

# Supprime le doublon unique détecté
df = df.drop_duplicates()



📊 Missing values (%):
provenance_codes_acces_exterieurs       77.4
appareils_conso_films_series            67.4
gratuit_ou_payant                       59.4
appareils_conso_musique_videos          43.2
frequence_conso_culturelle              42.4
type_conso_legale_ou_illegale           42.4
statut_emploi                           36.9
depense_mensuelle_culturelle            36.0
acces_services_payants                  36.0
utilisation_applis_crackees             29.8
utilisation_telechargement_streaming    29.8
profession_principale                   16.2
evolution_conso_legale                   8.8
nb_enfants                               0.0
taille_foyer                             0.0
reglages_dns                             0.0
sexe                                     0.0
utilisation_vpn                          0.0
age                                      0.0
frequence_internet                       0.0
situation_personnelle                    0.0
type_agglomeration              

In [17]:
# ============================================================
# 4️⃣ Nettoyage général
# ============================================================
df.columns = [c.lower().replace(" ", "_") for c in df.columns]
df["age"] = pd.to_numeric(df["age"], errors="coerce").astype("Int64")

# Nettoyage texte
for c in df.select_dtypes("object").columns:
    df[c] = df[c].astype(str).str.strip()


In [43]:
df.to_excel("barometre_clean.xlsx", index=False, engine="openpyxl")